In [ ]:
from preprocessing import getdataset,character_to_number, number_to_character

MIN_FREQ = 20

dataset , table, MAX_LEN = getdataset()
table.crop(MIN_FREQ)
print(table.vocab_size)


2024-12-27 00:45:21.480136: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-27 00:45:21.488777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-27 00:45:21.498392: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-27 00:45:21.501238: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-27 00:45:21.509046: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
from network import make_model

EMBED_DIM = 128
NUM_HEADS = 8
NUM_BLOCS = 16  
model = make_model(MAX_LEN, table.vocab_size, EMBED_DIM, NUM_HEADS,NUM_BLOCS)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 62)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding_1  │ (None, 62, 128)        │       826,112 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_16            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_17            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_18            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_19            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_20            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_21            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_22            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_23            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_24            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_25            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_26            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_27            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_28            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_29            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_30            │ (None, 62, 128)        │       544,512 │
│ (TransformerBlock)              │                        │             

 Total params: 9,675,116 (36.91 MB)

 Trainable params: 9,675,116 (36.91 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])


In [ ]:
X = tf.convert_to_tensor(dataset)
Y = np.zeros(X.shape).astype(int)
Y[:,0:-1] = np.array(X[:,1:])
Y = tf.convert_to_tensor(Y)
print(X)
print(Y)
#Y = tf.one_hot(Y, depth=table.vocab_size)
# change X to one_hot_vectors

print("pourcentage sur la prédiction de que des 0 : ",np.sum(X == 0)/Y.shape[0]/Y.shape[1]*100)

tf.Tensor(
[[    2     3     4 ...     0     0     0]
 [    2    10    11 ...     0     0     0]
 [    2    10    11 ...     0     0     0]
 ...
 [    2 78691     1 ...     0     0     0]
 [    2 78692     1 ...     0     0     0]
 [    2 78693     1 ...     0     0     0]], shape=(43352, 62), dtype=int32)
tf.Tensor(
[[    3     4     5 ...     0     0     0]
 [   10    11    26 ...     0     0     0]
 [   10    11    41 ...     0     0     0]
 ...
 [78691     1     0 ...     0     0     0]
 [78692     1     0 ...     0     0     0]
 [78693     1     0 ...     0     0     0]], shape=(43352, 62), dtype=int64)
pourcentage sur la prédiction de que des 0 :  63.60669448594849


In [ ]:
# example : 
tweet = np.array(X[0])

print("tweet : ",[number_to_character(x,table) for x in tweet])

tweet :  ['<sos>', 'be', 'sure', 'to', 'tune', 'in', 'and', 'watch', 'donald', 'trump', 'on', 'late', 'night', 'with', 'david', 'letterman', 'as', 'he', 'presents', 'the', 'top', 'ten', 'list', 'tonight!', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
history = model.fit(X, Y, epochs=30, batch_size=128,verbose=1)
model.save("model.h5")

Epoch 1/30
339/339 ━━━━━━━━━━━━━━━━━━━━ 64s 188ms/step - loss: nan - sparse_categorical_accuracy: 0.6518
Epoch 2/30
339/339 ━━━━━━━━━━━━━━━━━━━━ 62s 182ms/step - loss: nan - sparse_categorical_accuracy: 0.6523
Epoch 3/30
339/339 ━━━━━━━━━━━━━━━━━━━━ 61s 180ms/step - loss: nan - sparse_categorical_accuracy: 0.6528
Epoch 4/30
339/339 ━━━━━━━━━━━━━━━━━━━━ 61s 181ms/step - loss: nan - sparse_categorical_accuracy: 0.6516
Epoch 5/30
339/339 ━━━━━━━━━━━━━━━━━━━━ 56s 164ms/step - loss: nan - sparse_categorical_accuracy: 0.6521
Epoch 6/30
339/339 ━━━━━━━━━━━━━━━━━━━━ 55s 161ms/step - loss: nan - sparse_categorical_accuracy: 0.6530
Epoch 7/30
 46/339 ━━━━━━━━━━━━━━━━━━━━ 57s 197ms/step - loss: nan - sparse_categorical_accuracy: 0.6511

KeyboardInterrupt: 

In [ ]:
model.save("model.h5")

In [ ]:

text_to_begin_with = "the".split(" ")
begining = [ table.table[i] for i in text_to_begin_with ]
nb_to_repeat = MAX_LEN - len(begining) -1
input = np.array([[2] + begining + [0] * nb_to_repeat])
exit = False
nb_iter = len(text_to_begin_with)+1
max_iter = MAX_LEN

print( "".join( [table.number_to_character( input[0,i] ) for i in range(len(input[0]))] ) )

while not exit:
    output = model.predict(input)

    input[0,nb_iter] = np.argmax(output[0,nb_iter-1])
    nb_iter += 1
    if nb_iter == max_iter:
        exit = True
    
print( "".join( [table.number_to_character( input[0,i] ) for i in range(len(input[0]))] ) )

<sos>the<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


2024-12-27 00:24:44.390892: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_42', 8 bytes spill stores, 8 bytes spill loads

2024-12-27 00:24:44.870770: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_61', 4 bytes spill stores, 4 bytes spill loads

2024-12-27 00:24:45.064764: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_59', 4 bytes spill stores, 4 bytes spill loads



1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━

10